In [37]:
import csv
import time
import requests
from bs4 import BeautifulSoup
from pattern.en import ngrams
from textblob import TextBlob
from textblob import classifiers
import pandas as pd

In [58]:
PATH1 = 'data/SentimentData/amazon_cells_labelled.txt'
PATH2 = 'data/SentimentData/imdb_labelled.txt'
PATH3 = 'data/SentimentData/yelp_labelled.txt'

amazon_data = pd.read_csv(PATH1, sep='\t', header=None, names=['sentence', 'sentiment'])
imdb_data = pd.read_csv(PATH2, sep='\t', header=None, names=['sentence', 'sentiment'])
yelp_data = pd.read_csv(PATH3, sep='\t', header=None, names=['sentence', 'sentiment'])

In [106]:
total_data = amazon_data
total_data = total_data.append(imdb_data, ignore_index=True)
total_data = total_data.append(yelp_data, ignore_index=True)

In [144]:
train_data_full = [tuple(x) for x in total_data.values]
train_data = train_data_full[:2000]
print(len(train_data))

2748
2000


In [145]:
print('training NLP classifier...')
classifier = classifiers.NaiveBayesClassifier(train_data)
classifier.train()
print('done')

training NLP classifier...
done


In [146]:
print('classifying...')
prob_dist = classifier.prob_classify("Stocks are terrible")
print('result')
print(prob_dist.prob(0))

classifying...
result
0.6754882030498195


In [147]:
classifier.accuracy(train_data_full[2000:])

0.7593582887700535

In [152]:
??search

In [167]:
from newsapi import NewsApiClient

key = '8b8428b3603940ef8485d39a6eb526f2'
news = NewsApiClient(api_key=key)

companies = {'Synchrony Financial': [],
            '3M': [],
            'Bayer': [],
            'Honeywell': []}

ARTICLE_COUNT = 5

for company in companies:
    query = company + " News"
    all_articles = news.get_everything(q=query, from_param='2019-02-01',to='2019-02-02', language='en', sort_by='relevancy')
    article_data = all_articles['articles']
    for i in range(ARTICLE_COUNT):
        try:
            titlePlusSummary = article_data[i]['title'] + '. ' + article_data[i]['description']
            companies[company].append(titlePlusSummary)
        except:
            break

In [168]:
print(companies)

{'Synchrony Financial': ["Synchrony's Investors Have 2 Big Reasons to Smile. The store credit card giant just gave investors some welcome news along with its fourth-quarter earnings.", "5 Top Stocks to Buy in February. Even after January's rally, these stocks look like bargains to our contributors.", 'Zacks.com featured expert Kevin Matras highlights: HD Supply, Synchrony Financial and Alaska Air. Zacks.com featured expert Kevin Matras highlights: HD Supply, Synchrony Financial and Alaska Air', "5 Top Stocks to Buy in February. Even after January's rally, these stocks look like bargains to our contributors.", 'Three Highlights from @CloudEXPO NY | #Cloud #CIO #DevOps #Blockchain #DataCenter #ArtificialIntelligence. The term "digital transformation" (DX) is being used by everyone for just about any company initiative that involves technology, the web, ecommerce, software, or even customer experience. While the term has certainly turned into a buzzword with a lot of hype,…'], '3M': ["3M 

In [170]:
# Open the choosen news articles and extract the title
title_to_polarity = {}
net_company_scores = {}
for company in companies:
    net_company_scores[company] = 0
    count = 0
    for summary in companies[company]:
        try:
            prob_1 = classifier.prob_classify(summary).prob(1)
            title_to_polarity[summary] = prob_1
            net_company_scores[company] += prob_1
        except:
            continue
    if count:
        net_company_scores[company] /= count

print(net_company_scores)

{'Synchrony Financial': 1.3697317812082912, '3M': 1.5567991879540197, 'Bayer': 3.6251427205105813, 'Honeywell': 2.003248803329603}


In [171]:
print(title_to_polarity)

{"Synchrony's Investors Have 2 Big Reasons to Smile. The store credit card giant just gave investors some welcome news along with its fourth-quarter earnings.": 0.5093374403944637, "5 Top Stocks to Buy in February. Even after January's rally, these stocks look like bargains to our contributors.": 0.004913103023015138, 'Zacks.com featured expert Kevin Matras highlights: HD Supply, Synchrony Financial and Alaska Air. Zacks.com featured expert Kevin Matras highlights: HD Supply, Synchrony Financial and Alaska Air': 0.8491737287107234, 'Three Highlights from @CloudEXPO NY | #Cloud #CIO #DevOps #Blockchain #DataCenter #ArtificialIntelligence. The term "digital transformation" (DX) is being used by everyone for just about any company initiative that involves technology, the web, ecommerce, software, or even customer experience. While the term has certainly turned into a buzzword with a lot of hype,…': 0.0013944060570735702, "3M Opens 2019 on an Unimpressive Note. There were some positives fr